## Application #1: nbconvert


## Additional resource: <br>"Writing (and publishing) a book written in Jupyter notebooks"


Excellent discussion of the nbconvert machinery yesterday by Andreas Mueller:  
[link to the video(once it exists)](https://conferences.oreilly.com/jupyter/jup-ny/public/schedule/detail/60087)

## Distinction between use cases for nbformat and nbconvert

`nbformat`: creating and validating notebooks 

`nbconvert`: manipulate existing notebooks.


## Nbconvert allows many different kinds of interactions

* converting notebooks into other formats

    * web-display: html, slides (with reveal.js)

    * publishable documents: LaTeX/PDF, ASCIIDoc

    * plain-text: rst, markdown

    * executable scripts: `.py` (and others if defined by the kernel)

* manipulating notebooks directly 
    * e.g., removing content
* executing notebooks from the command-line

## Overall architecture

* Exporters
* Preprocessors
* Templates (and filters)
* Writers
* Postprocessors

## Exporters

Orchestration layer.

Exporters often are used to define the Preprocessors, Templates and filters that are to be used in a notebook.

## Preprocessors

Notebook to notebook transforms.

This is how execution is enabled through the `ExecutePreprocessor`.

## 